<a href="https://colab.research.google.com/github/NikhilaArutla/Nikhila_INFO5731_Spring2025/blob/main/Arutla_Nikhila_Week_12_Inclass_Activity_absa_hf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "yangheng/deberta-v3-base-absa-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

def predict(text, aspects):
  sentiment_aspect = {}
  for aspect in aspects:
    inputs = tokenizer(text, aspect, return_tensors="pt")

    with torch.inference_mode():
      outputs = model(**inputs)

    scores = F.softmax(outputs.logits[0], dim=-1)
    label_id = torch.argmax(scores).item()
    sentiment_aspect[aspect] = (model.config.id2label[label_id], scores[label_id].item())
  return sentiment_aspect


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/372 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

In [2]:
aspects = ["food", "service"]
text = "The food was great but the service was terrible."
predict(text, ['service'])

{'service': ('Negative', 0.9935430288314819)}

In [3]:
predict('The screen is too large to fit in my pocket, but the battery life is great', ['battery life', 'screen'])

{'battery life': ('Positive', 0.9977203011512756),
 'screen': ('Negative', 0.9851062297821045)}

In [4]:
predict('The screen is too large to fit in my pocket, but the battery life is great', ['screen'])

{'screen': ('Negative', 0.9851062297821045)}

In [5]:
text = """I just want to give a shout out for Garmin phone support. I was having trouble getting GPS to find satellites this summer,
and it was losing track mid swim. Mary Beth at Garmin support walked me through several paths to finally get the find time down to under a minute.
She was patient and seemed genuinely interested in solving my problem -- a rare event from support lines these days.
I'm still loving my Swim 2 after nearly a year."""
predict(text, ['Garmin phone support'])

{'Garmin phone support': ('Positive', 0.9852809309959412)}

In [6]:
text = """GPS takes awhile before connecting sometimes, like 6-10min. Which is kinda really long when seating in the car ready to go...
GPS does glitch badly sometimes, but it is pretty rare and you know right away when something is wrong.
Disappointed that it does not recognize your swim exercice more accurately. Watch does not give maximum speed for any mode,
you can only find out your top speed once you downloaded the data to your phone and use the app."""
predict(text, ['GPS', 'accuracy', 'watch'])

{'GPS': ('Negative', 0.9745671153068542),
 'accuracy': ('Negative', 0.9755191206932068),
 'watch': ('Negative', 0.9889798164367676)}